# IMDB Person Bio 

In [1]:
# Configure Environment 
from  prod.data.settings import * 
print("Platform: " , PLATFORM)
print("Data directory : ", os.getcwd())


Platform:  dev
Data directory :  /home/jovyan/dev/data


In [2]:

import os
import sys
import time
import logging
import numpy as np
import pandas as pd
from imdb import IMDb
import json
from datetime import datetime
# create an instance of the IMDb class
ia = IMDb()


In [3]:
topActors = pd.read_csv('./top100Actors.txt', 
                     sep='\t',
                     header=0, 
                    dtype=str)
print(type(topActors))
topActors.head(15)

<class 'pandas.core.frame.DataFrame'>


,Rank,First Name,Last Name,Domestic Box Office,Movies,Average
0,1,Robert,Downey Jr.,"$5,374,149,735",43,"$124,980,226"
1,2,Samuel L.,Jackson,"$5,234,777,328",60,"$87,246,289"
2,3,Scarlett,Johansson,"$4,859,403,632",28,"$173,550,130"
3,4,Tom,Hanks,"$4,692,293,839",51,"$92,005,762"
4,5,Bradley,Cooper,"$4,263,310,067",26,"$163,973,464"
5,6,Tom,Cruise,"$3,937,740,350",42,"$93,755,723"
6,7,Zoe,Saldana,"$3,932,967,185",26,"$151,267,969"
7,8,Chris,Hemsworth,"$3,915,069,280",22,"$177,957,695"
8,9,Chris,Pratt,"$3,866,023,741",17,"$227,413,161"
9,10,Harrison,Ford,"$3,850,910,074",40,"$96,272,752"


In [4]:
actorsDF = topActors[['Rank','First Name','Last Name']]
actorsDF['Full Name']= actorsDF['First Name']+' '+actorsDF['Last Name']
actorsDF = actorsDF.drop(['First Name','Last Name'], axis=1)
actors = actorsDF.values.tolist()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Search by actor and return biographic data

In [5]:
# search for a person name
def imdbSearchPerson(match_actor):
    persons = ia.search_person(match_actor)
    for person in persons:
        if person['name'] == match_actor:
            actorID = person.getID()
            return actorID
        else:
            print(persons[0])

In [6]:
def imdbPersonBio(actorID):
    personBio=ia.get_person(actorID, info=['biography'])
    personBioDic ={}
    personBioKeys =personBio.infoset2keys

    for key in personBioKeys['biography']:
        try:
            # TODO : clean out special IMDB chars
            personBioDic[key]= personBio.data[key]
        except:
            print('Error adding key :', key)

    return personBioDic


In [7]:
for actor in actors:
    try:
        _actor_id = actor[0]
        actorFullName=actor[1]
        actorID = imdbSearchPerson(actorFullName)
        personBio = imdbPersonBio(actorID)
        personBio.update( {'_actorID': _actor_id,  '_source': 'IMDB', 'actorID': actorID, 'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")})

        f = open('./person/imdb/imdb_p_'+_actor_id+'.json', 'w+')
        f.write(json.dumps(personBio))
        f.close()
    except:
        print('Error',  _actor_id)

Michael Keaton


# Retrieve Movie info from IMDB 

In [8]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import http.client
import json
import logging
logger = logging.getLogger('movieInfo')
logger.setLevel(logging.INFO)

In [9]:
path_to_json = './cast/themoviedb/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
len(json_files)
df = pd.concat([pd.read_json(f , dtype=str) for f in json_files], ignore_index = True)
df

,_actorID,_source,actor,actorID,character,characterID,created_utc_dt,movieID,movieTitle
0,5,themoviedb,Bradley Cooper,51329,Steve,52fe4473c3a368484e023c09,"09/14/2019, 12:47:01",23706,All About Steve
1,5,themoviedb,Bradley Cooper,51329,Holden Wilson,52fe44eb9251416c910216df,"09/14/2019, 12:47:01",32856,Valentine's Day
2,5,themoviedb,Bradley Cooper,51329,Lt. Templeton 'Faceman' Peck,52fe456a9251416c91031899,"09/14/2019, 12:47:01",34544,The A-Team
3,5,themoviedb,Bradley Cooper,51329,Douglas J. Ames,52fe458ec3a368484e062293,"09/14/2019, 12:47:01",28355,Case 39
4,5,themoviedb,Bradley Cooper,51329,Phil Wenneck,52fe46b8c3a36847f810e649,"09/14/2019, 12:47:01",45243,The Hangover Part II
5,5,themoviedb,Bradley Cooper,51329,Phil Wenneck,52fe479a9251416c7509f5eb,"09/14/2019, 12:47:01",18785,The Hangover
6,5,themoviedb,Bradley Cooper,51329,Eddie Morra,52fe4809c3a36847f81552c9,"09/14/2019, 12:47:01",51876,Limitless
7,5,themoviedb,Bradley Cooper,51329,Rory Jansen,52fe48b49251416c91095939,"09/14/2019, 12:47:01",83686,The Words
8,5,themoviedb,Bradley Cooper,51329,Avery Cross,52fe49dd9251416c750d5ea5,"09/14/2019, 12:47:01",97367,The Place Beyond the Pines
9,5,themoviedb,Bradley Cooper,51329,Phil,52fe4ab6c3a36847f81ddb37,"09/14/2019, 12:47:01",109439,The Hangover Part III


In [10]:
movieDF = df[['movieID','movieTitle']]
movieDF.drop_duplicates(keep = False, inplace = True) 
movieDF.head()

movieDF.to_csv('./movies/movie_list.tsv', sep='\t')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
from imdb import IMDb
# create an instance of the IMDb class
ia = IMDb()
print(ia.get_movie_infoset())
print(ia.get_person_infoset())
print(ia.get_company_infoset())


['airing', 'akas', 'alternate versions', 'awards', 'connections', 'crazy credits', 'critic reviews', 'episodes', 'external reviews', 'external sites', 'faqs', 'full credits', 'goofs', 'keywords', 'locations', 'main', 'misc sites', 'news', 'official sites', 'parents guide', 'photo sites', 'plot', 'quotes', 'release dates', 'release info', 'reviews', 'sound clips', 'soundtrack', 'synopsis', 'taglines', 'technical', 'trivia', 'tv schedule', 'video clips', 'vote details']
['awards', 'biography', 'filmography', 'genres links', 'keywords links', 'main', 'news', 'official sites', 'other works', 'publicity']
['main']


# Extract IMDB Movie metadata 

In [12]:
main_attributes= ['genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'certificates', 'original air date',  'rating',  'votes',  'cover url',  'plot outline',  'languages', 'title', 'year',  'kind']
person_attributes = ['cast','directors',  'writers',  'producers', 'composers']
company_atttributes = ['production companies',  'distributors','special effects']
object_attributes = set(person_attributes+ company_atttributes )
person_nodes=[]
company_nodes=[]
person_edges=[]
company_edges=[]
user_reviews=[]
from collections import defaultdict
movie_person = defaultdict(list)
movie_companies = defaultdict(list)

for index, row in movieDF.iterrows():
        movieInfo={}
        movieID = row['movieID']
        movieTitle = row['movieTitle']
        movies = ia.search_movie(movieTitle)
        logger.warning('Processing IMDB movie : '+ movieTitle )
        try:
            
            if movies:
                IMDB_movieID = movies[0].movieID
                movie = ia.get_movie(IMDB_movieID)
                movieInfo.update( {'_id': movieID, 'source': 'IMDB', 'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")})

                # Return available movie attributes 
                movie_keys= movie.infoset2keys
                movie_keys=set(movie_keys['main'])
                matched_keys = movie_keys.difference(object_attributes)

                #= set(main_attributes).intersection(set(movie_keys['main']))


                for key in matched_keys:
                    if type(movie.data[key]) == list:
                        if str(type(movie.data[key][0])).find('imdb.') != -1:
                            pass
                        else:
                            movieInfo[key]= movie.data[key]
                    else:
                        movieInfo[key]= movie.data[key]


                for company_info in set(movie_keys).intersection(set(company_atttributes)) :
                    for info in movie.data[company_info]:
                        movie_companies[company_info]=  list(set( movie_companies[company_info]+ [info['name']] ))  

                for person_info in set(movie_keys).intersection(set(person_attributes)):
                    for persons in movie.data[person_info]:
                        try:
                            movie_person[person_info] = list(set( movie_person[person_info] + [persons['name']] ))   
                        except:
                            continue
                movieInfo['cast'] = movie_person
                movieInfo['companies'] = movie_companies

                movieInfo['votes']= ia.get_movie(movieID, info=['vote details']).data

                #print(movieInfo)
                movie_file = open('./movies/imdb/imdb_m_'+str(movieID)+'.json',"w+")
                movie_file.write(json.dumps(movieInfo))
                movie_file.close()    
                
        except Exception as e:
            logger.error('ERROR processing IMDB Movie: '+ str(e))


Processing IMDB movie : Case 39
Processing IMDB movie : The Hangover Part II
Processing IMDB movie : The Hangover
Processing IMDB movie : The Place Beyond the Pines
Processing IMDB movie : The Hangover Part III
Processing IMDB movie : Hit & Run
Processing IMDB movie : Burnt
Processing IMDB movie : 10 Cloverfield Lane
Processing IMDB movie : Hurricane of Fun: The Making of Wet Hot
Processing IMDB movie : War Dogs
Processing IMDB movie : A Star Is Born
Processing IMDB movie : Making a Scene
Processing IMDB movie : The Road to Stardom: The Making of A Star is Born
Processing IMDB movie : Baby, Baby, Baby
Processing IMDB movie : Marmaduke
Processing IMDB movie : Hall Pass
Processing IMDB movie : Cars 2
Processing IMDB movie : Are You Here
Processing IMDB movie : Free Birds
2019-09-14 13:05:46,723 CRITICAL [imdbpy] /opt/conda/lib/python3.6/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title

Processing IMDB movie : From the Ashes: Post-Production and Release of 'The Expendables'
Processing IMDB movie : Creed II
Processing IMDB movie : Backtrace
Processing IMDB movie : Stallone, profession héros
Processing IMDB movie : The Last Stand
Processing IMDB movie : Terminator Genisys
Processing IMDB movie : Sabotage
2019-09-14 13:09:32,676 CRITICAL [imdbpy] /opt/conda/lib/python3.6/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0144336/ratings', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: 'Not Found'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/imdb/parser/http/__init__.py", line 219, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/conda/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/opt/conda/lib/python3.6/urllib/request.p

Processing IMDB movie : Family Guy Presents: It's a Trap!
ERROR processing IMDB Movie: Object of type 'Movie' is not JSON serializable
Processing IMDB movie : Pirates of the Caribbean: On Stranger Tides
Processing IMDB movie : The Rum Diary
Processing IMDB movie : The Imaginarium of Doctor Parnassus
Processing IMDB movie : Alice in Wonderland
Processing IMDB movie : Dark Shadows
Processing IMDB movie : Rango
Processing IMDB movie : The Lone Ranger
Processing IMDB movie : Alice Through the Looking Glass
Processing IMDB movie : Yoga Hosers
Processing IMDB movie : Tusk
Processing IMDB movie : Sherlock Gnomes
Processing IMDB movie : Donald Trump's The Art of the Deal: The Movie
Processing IMDB movie : Fantastic Beasts and Where to Find Them
Processing IMDB movie : Fantastic Beasts: The Crimes of Grindelwald
Processing IMDB movie : City of Lies
Processing IMDB movie : The Black Ghiandola
2019-09-14 13:14:33,325 CRITICAL [imdbpy] /opt/conda/lib/python3.6/site-packages/imdb/_exceptions.py:34:

Processing IMDB movie : 35 and Ticking
Processing IMDB movie : Think Like a Man
Processing IMDB movie : Chocolate Sundaes Comedy Show: Live On Sunset Strip!
Processing IMDB movie : Kevin Hart: Laugh at My Pain
Processing IMDB movie : Exit Strategy
Processing IMDB movie : Backstage at Budz House
Processing IMDB movie : Let Go
Processing IMDB movie : Kevin Hart: Let Me Explain
Processing IMDB movie : Think Like a Man Too
Processing IMDB movie : About Last Night
Processing IMDB movie : James Brown - The Man, The Music & The Message
Processing IMDB movie : The Wedding Ringer
Processing IMDB movie : Kevin Hart: Serve and Protect
Processing IMDB movie : The Secret Life of Pets
Processing IMDB movie : Keith Robinson: Back of the Bus Funny
Processing IMDB movie : Ride Along 2
Processing IMDB movie : Dying Laughing
Processing IMDB movie : Ride Along
Processing IMDB movie : Captain Underpants: The First Epic Movie
Processing IMDB movie : Kevin Hart Presents: Plastic Cup Boyz
Processing IMDB movi

Processing IMDB movie : Eating You Alive
Processing IMDB movie : From Paris with Love
Processing IMDB movie : Savages
Processing IMDB movie : The Forger
Processing IMDB movie : Criminal Activities
Processing IMDB movie : I Am Wrath
Processing IMDB movie : Close Encounters with Vilmos Zsigmond
Processing IMDB movie : Life on the Line
Processing IMDB movie : Gotti
Processing IMDB movie : Stayin' Alive: A Grammy Salute to the Music of the Bee Gees
Processing IMDB movie : Speed Kills
Processing IMDB movie : Trading Paint
Processing IMDB movie : The Grease Story
Processing IMDB movie : Saturday Night Fever: The Ultimate Disco Movie
Processing IMDB movie : Urban Cowboy: The Rise and Fall of Gilley's
Processing IMDB movie : Country Strong
Processing IMDB movie : Glee: The Concert Movie
Processing IMDB movie : Man in Red Bandana
Processing IMDB movie : Coldplay: A Head Full of Dreams
Processing IMDB movie : Everything Must Go
Processing IMDB movie : Land of the Lost
Processing IMDB movie : Cas

...

2019-09-14 13:48:04,028 CRITICAL [imdbpy] /opt/conda/lib/python3.6/site-packages/imdb/__init__.py:737: caught an exception retrieving or parsing "vote details" info set for mopID "0191502" (accessSystem: http)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/imdb/parser/http/__init__.py", line 219, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/conda/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/opt/conda/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/opt/conda/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/opt/conda/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/opt/conda/lib/python3.6/urllib/request.py", line 650, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP E

Processing IMDB movie : Vacation
Processing IMDB movie : 12 Strong
Processing IMDB movie : Team Thor: Part 2
Processing IMDB movie : Bad Times at the El Royale
Processing IMDB movie : Heroes and Demons
ERROR processing IMDB Movie: Object of type 'Movie' is not JSON serializable
Processing IMDB movie : Deep in the Valley
Processing IMDB movie : Take Me Home Tonight
Processing IMDB movie : Jennifer's Body
Processing IMDB movie : Zero Dark Thirty
Processing IMDB movie : Delivery Man
Processing IMDB movie : Jurassic World
Processing IMDB movie : Timms Valley
Processing IMDB movie : The Kid
Processing IMDB movie : Jurassic World: Fallen Kingdom
Processing IMDB movie : Emmet's Holiday Party: A Lego Movie Short
Processing IMDB movie : Parks and Recreation: 10th Anniversary Reunion at PALEYFEST 2019
Processing IMDB movie : Cherrybomb
Processing IMDB movie : Into the White
Processing IMDB movie : CBGB
Processing IMDB movie : Postman Pat: The Movie
Processing IMDB movie : Moonwalkers
Processing 

Processing IMDB movie : WWE: Batista – The Animal Unleashed
Processing IMDB movie : Spectre
Processing IMDB movie : L.A. Slasher
Processing IMDB movie : Kickboxer: Vengeance
Processing IMDB movie : Satan's Prison: The Anthology of the Elimination Chamber
Processing IMDB movie : Master Z: Ip Man Legacy
Processing IMDB movie : Randy Orton: RKO Outta Nowhere
Processing IMDB movie : WWE: The Undertaker's Deadliest Matches
Processing IMDB movie : Bushwick
Processing IMDB movie : Enter the Warriors Gate
Processing IMDB movie : Hotel Artemis
Processing IMDB movie : 2048: Nowhere to Run
Processing IMDB movie : My Son, My Son, What Have Ye Done
Processing IMDB movie : Shawn Michaels: The Showstopper Unreleased
Processing IMDB movie : WWE WrestleMania 35
ERROR processing IMDB Movie: Object of type 'Movie' is not JSON serializable
Processing IMDB movie : Rock of Ages
Processing IMDB movie : Jack Reacher
Processing IMDB movie : The Mummy
Processing IMDB movie : Edge of Tomorrow
Processing IMDB mov

Processing IMDB movie : High-Rise
Processing IMDB movie : Out of Time
Processing IMDB movie : Stars In Shorts
Processing IMDB movie : Early Man
Processing IMDB movie : Leading Lady Parts
Processing IMDB movie : The Man Who Smiled
Processing IMDB movie : Creating a King: Realizing an Icon
Processing IMDB movie : The House is Alive: Constructing 'Crimson Peak'
Processing IMDB movie : Loup
Processing IMDB movie : Porn in the Hood
Processing IMDB movie : Hacker's Game
Processing IMDB movie : Paris or Perish
Processing IMDB movie : Radiostars
Processing IMDB movie : Silhouettes
Processing IMDB movie : Love Lasts Three Years
Processing IMDB movie : Delicacy
Processing IMDB movie : Borderline
Processing IMDB movie : Sleepless Night
Processing IMDB movie : Newness
Processing IMDB movie : A. I. Tales
Processing IMDB movie : Time of Day
Processing IMDB movie : Hacker's Game Redux
Processing IMDB movie : Dante's Inferno: An Animated Epic
Processing IMDB movie : Sushi Girl
Processing IMDB movie : 

Processing IMDB movie : G.I. Joe: The Rise of Cobra
Processing IMDB movie : The Dilemma
Processing IMDB movie : The Eagle
Processing IMDB movie : Morgan and Destiny's Eleventeenth Date: The Zeppelin Zoo
Processing IMDB movie : The Son of No One
Processing IMDB movie : Jupiter Ascending
Processing IMDB movie : Magic Mike
Processing IMDB movie : Side Effects
Processing IMDB movie : White House Down
Processing IMDB movie : 22 Jump Street
Processing IMDB movie : Magic Mike XXL
Processing IMDB movie : The Vow
Processing IMDB movie : The Lego Batman Movie
Processing IMDB movie : Kingsman: The Golden Circle
Processing IMDB movie : Smallfoot
Processing IMDB movie : Crazy on the Outside
Processing IMDB movie : I Am Comic
Processing IMDB movie : The Six Wives of Henry Lefay
Processing IMDB movie : Pixar 25 Magic Moments
Processing IMDB movie : Chimpanzee
Processing IMDB movie : The Penguin King
Processing IMDB movie : 3 Geezers!
Processing IMDB movie : The Atheist Delusion
Processing IMDB movie 

In [13]:
# Copy models up to S3 bucket
bucket = os.popen('aws ssm get-parameter --name jupyterBucketName-' + PLATFORM + ' --with-decryption | jq -r .Parameter.Value').read()
print(bucket)
executable = 'aws s3 sync ./movies/ s3://' + bucket.strip() + '/movies/ '
print(executable)
myCmd = os.popen(executable).read()
print(myCmd)

bdso-celebrity-content-dev-vpc-05447d97de18e28f8

aws s3 sync ./movies/ s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/movies/ 
Completed 256.0 KiB/~7.9 MiB (3.2 MiB/s) with ~12 file(s) remaining (calculating...)
Completed 512.0 KiB/~7.9 MiB (5.9 MiB/s) with ~12 file(s) remaining (calculating...)
Completed 768.0 KiB/~7.9 MiB (6.2 MiB/s) with ~12 file(s) remaining (calculating...)
Completed 1.0 MiB/~7.9 MiB (7.7 MiB/s) with ~12 file(s) remaining (calculating...)  
Completed 1.2 MiB/~7.9 MiB (8.5 MiB/s) with ~12 file(s) remaining (calculating...)  
Completed 1.5 MiB/~7.9 MiB (10.0 MiB/s) with ~12 file(s) remaining (calculating...) 
Completed 1.8 MiB/~7.9 MiB (11.6 MiB/s) with ~12 file(s) remaining (calculating...) 
Completed 2.0 MiB/~7.9 MiB (13.2 MiB/s) with ~12 file(s) remaining (calculating...) 
Completed 2.2 MiB/~7.9 MiB (14.7 MiB/s) with ~12 file(s) remaining (calculating...) 
Completed 2.5 MiB/~7.9 MiB (15.6 MiB/s) with ~12 file(s) remaining (calculating...) 
Completed 2.8 M